<a href="https://colab.research.google.com/github/anabiancachiricu/BigDataProject/blob/main/BigDataProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Proiect Big data - Chiricu Ana Bianca, grupa 405

1. Introducere

Am ales setul de date Netflix Movies and TV Shows care reprezinta o lista cu toate filmele si serialele disponibile pe platforma de streaming Netflix la mmomentul crearii setului de date, impreuna cu detalii despre fiecare, cum ar fi durata, anul lansarii, distributia sau regizorii.



Link data set: https://www.kaggle.com/datasets/shivamb/netflix-shows

2.

In [1]:
!pip install PySpark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PySpark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=0b94200bd67ab4483b9d7aa42c4ad2e46529b66c14bfd4e623691b1b663a451f
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built PySpark


In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import seaborn as sns
import matplotlib.pyplot as plt
spark = SparkSession.builder.getOrCreate()

In [3]:
dataframe = spark.read.csv('sample_data/netflix_titles.csv', inferSchema=True, header=True)
dataframe.show(10)

+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|     Kirsten Johnson|                null|       United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|                null|Ama Qamata, Khosi...|        South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglan

In [4]:
dataframe.registerTempTable('NetflixTable')

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


Verificarea si eliminarea inregistrarilor care au alt tip in afara de tv show si movie.

In [9]:
df2= spark.sql("Select distinct (type) from NetflixTable" )
df2.show()

+-------------+
|         type|
+-------------+
|         null|
|      TV Show|
|        Movie|
|William Wyler|
+-------------+



In [13]:
df3= spark.sql("Select * from NetflixTable where type in ('TV Show', 'Movie') ")

In [19]:
df3.show(10)

+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|     Kirsten Johnson|                null|       United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|                null|Ama Qamata, Khosi...|        South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglan

Eliminarea inregistrarilor care nu au an de lansare.

In [18]:
df4= spark.sql("Select * from NetflixTable where type in ('TV Show', 'Movie') and release_year like ('____') order by release_year ")
df4.show()

+-------+-------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+------+--------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|       date_added|release_year|rating|duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+------+--------+--------------------+--------------------+
|  s4251|TV Show|Pioneers: First W...|                null|                null|                null|December 30, 2018|        1925| TV-14|1 Season|            TV Shows|This collection r...|
|  s7791|  Movie|      Prelude to War|         Frank Capra|                null|       United States|   March 31, 2017|        1942| TV-14|  52 min|Classic Movies, D...|Frank Capra's doc...|
|  s8206|  Movie|The Battle of Midway|       

In [24]:
df4.registerTempTable('NetflixTable')

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [26]:
df5= spark.sql("Select * from NetflixTable where director is not null and cast is not null and country is not null and date_added is not null and release_year is not null and rating is not null and duration is not null and listed_in is not null and description is not null ")
df5.show()

+-------+-----+--------------------+--------------------+--------------------+--------------------+-----------------+------------+------+--------+--------------------+--------------------+
|show_id| type|               title|            director|                cast|             country|       date_added|release_year|rating|duration|           listed_in|         description|
+-------+-----+--------------------+--------------------+--------------------+--------------------+-----------------+------------+------+--------+--------------------+--------------------+
|  s8206|Movie|The Battle of Midway|           John Ford|Henry Fonda, Jane...|       United States|   March 31, 2017|        1942| TV-14|  18 min|Classic Movies, D...|Director John For...|
|  s8641|Movie|    Tunisian Victory|Frank Capra, John...|    Burgess Meredith|United States, Un...|   March 31, 2017|        1944| TV-14|  76 min|Classic Movies, D...|British and Ameri...|
|  s7220|Movie|Know Your Enemy -...|Frank Capra, Jori..